<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
#!pip3 install twilio

In [1]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [2]:
query = 'Pichilemu'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=d62a0e43d65440058f2165701231306&q=Pichilemu&days=1&aqi=no&alerts=no'

In [3]:
response = requests.get(url_clima).json()

In [4]:
response

{'location': {'name': 'Pichilemu',
  'region': 'Valparaiso',
  'country': 'Chile',
  'lat': -32.38,
  'lon': -71.13,
  'tz_id': 'America/Santiago',
  'localtime_epoch': 1686707917,
  'localtime': '2023-06-13 21:58'},
 'current': {'last_updated_epoch': 1686707100,
  'last_updated': '2023-06-13 21:45',
  'temp_c': 5.7,
  'temp_f': 42.3,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 3.8,
  'wind_kph': 6.1,
  'wind_degree': 235,
  'wind_dir': 'SW',
  'pressure_mb': 1020.0,
  'pressure_in': 30.11,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 63,
  'cloud': 0,
  'feelslike_c': 4.5,
  'feelslike_f': 40.1,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 8.1,
  'gust_kph': 13.0},
 'forecast': {'forecastday': [{'date': '2023-06-13',
    'date_epoch': 1686614400,
    'day': {'maxtemp_c': 17.8,
     'maxtemp_f': 64.0,
     'mintemp_c': 2.4,
     'mintemp_f': 36.3,
     'avgtem

In [5]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [6]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [7]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [8]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-13'

In [9]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1] #fecha

'01:00'

In [10]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #hora

1

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #Condition

'Clear'

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

2.4

In [13]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [14]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

# Dataframe

In [17]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [18]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|████████████████████████████████████████| 24/24 [00:00<00:00, 47215.43it/s]


In [26]:
datos[17]

('2023-06-13', 17, 'Partly cloudy', 10.4, 0, 0)

In [30]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-06-13,0,Clear,2.4,0,0
1,2023-06-13,1,Clear,2.6,0,0
2,2023-06-13,2,Clear,5.6,0,0
3,2023-06-13,3,Clear,6.4,0,0
4,2023-06-13,4,Clear,6.0,0,0
5,2023-06-13,5,Clear,4.8,0,0
6,2023-06-13,6,Clear,4.8,0,0
7,2023-06-13,7,Clear,4.8,0,0
8,2023-06-13,8,Sunny,5.2,0,0
9,2023-06-13,9,Sunny,9.1,0,0


In [33]:
#data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [42]:
df[df['Condicion']=='Sunny']

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
8,2023-06-13,8,Sunny,5.2,0,0
9,2023-06-13,9,Sunny,9.1,0,0
10,2023-06-13,10,Sunny,11.1,0,0
11,2023-06-13,11,Sunny,14.4,0,0
12,2023-06-13,12,Sunny,16.2,0,0
13,2023-06-13,13,Sunny,17.8,0,0
14,2023-06-13,14,Sunny,16.8,0,0
15,2023-06-13,15,Sunny,15.3,0,0
16,2023-06-13,16,Sunny,14.1,0,0


In [44]:
df_rain =  df[(df['Condicion']=='Sunny') & (df['Hora']>6) & (df['Hora']< 23)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [45]:
df_rain

,Condicion
Hora,
8,Sunny
9,Sunny
10,Sunny
11,Sunny
12,Sunny
13,Sunny
14,Sunny
15,Sunny
16,Sunny


# Mensaje - Template

In [46]:
'\nHola! \n\n\n El pronostico Sunny del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico Sunny del tiempo hoy 2023-06-13 en Pichilemu es : \n\n\n      Condicion\nHora          \n8        Sunny\n9        Sunny\n10       Sunny\n11       Sunny\n12       Sunny\n13       Sunny\n14       Sunny\n15       Sunny\n16       Sunny'

In [47]:
PHONE_NUMBER

'+1 430 245 7556'

# Mensaje SMS desde Twilio

In [48]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico Sunny de hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+56966724185'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM0afb36dd9192276649cfd5238306b8ab


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />